<a href="https://colab.research.google.com/github/Ucchwas/ECG-Classification-using-ML-models/blob/main/Hybrid_Multiclass_With%20GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical
from sklearn.utils import class_weight
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/mitbih_train.csv')

In [4]:
df.drop('1',axis='columns',inplace=True)
df.fillna(0, inplace=True)
X = df.drop('Label1',axis='columns')
y = testLabels = df.Label1.astype(np.float32)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2,train_size=0.8, random_state=15, stratify=y)
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix , classification_report

In [6]:
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, LSTM, BatchNormalization, Reshape
from keras.models import Model
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report
from keras.layers import concatenate

In [16]:
def CNN_RNN(X_train, y_train, X_test, y_test, loss, weights):
    im_shape = (X_train.shape[1], 1)
    inputs_cnn = Input(shape=im_shape, name='inputs_cnn')

    # Convolutional layers
    conv1_1 = Conv1D(64, 6, activation='relu', input_shape=im_shape)(inputs_cnn)
    conv1_1 = BatchNormalization()(conv1_1)
    pool1 = MaxPooling1D(pool_size=3, strides=2, padding="same")(conv1_1)

    conv2_1 = Conv1D(64, 3, activation='relu')(pool1)
    conv2_1 = BatchNormalization()(conv2_1)
    pool2 = MaxPooling1D(pool_size=2, strides=2, padding="same")(conv2_1)

    conv3_1 = Conv1D(32, 3, activation='relu')(pool2)
    conv3_1 = BatchNormalization()(conv3_1)
    pool3 = MaxPooling1D(pool_size=2, strides=2, padding="same")(conv3_1)

    flatten = Flatten()(pool3)
    # Reshape the output of Flatten to be 3D
    reshaped = Reshape((1, -1))(flatten)  # Reshape to (batch_size, 1, features)


    # Recurrent layer (LSTM)
    lstm_layer = LSTM(32, return_sequences=False)(reshaped)

    # Fully connected layers
    dense_end1 = Dense(32, activation='relu')(lstm_layer)
    dense_end2 = Dense(16, activation='relu')(dense_end1)
    main_output = Dense(5, activation='sigmoid', name='main_output')(dense_end2)

    model = Model(inputs=inputs_cnn, outputs=main_output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    if weights == -1:
        model.fit(X_train, y_train, epochs=50)
    else:
        model.fit(X_train, y_train, epochs=50, class_weight=weights)

    print(model.evaluate(X_test, y_test))

    y_preds = model.predict(X_train)
    y_preds = (y_preds > 0.5).astype(int)

    print("Classification Report: \n", classification_report(y_train, y_preds))
    return model, y_preds


In [8]:
from keras.utils import to_categorical

# Convert y_train to one-hot encoding
y_train = to_categorical(y_train, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

In [9]:
model,history=CNN_RNN(X_train,y_train,X_test,y_test, 'binary_crossentropy', -1)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs_cnn (InputLayer)     [(None, 186, 1)]          0         
                                                                 
 conv1d (Conv1D)             (None, 181, 64)           448       
                                                                 
 batch_normalization (Batch  (None, 181, 64)           256       
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 91, 64)            0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 89, 64)            12352     
                                                                 
 batch_normalization_1 (Bat  (None, 89, 64)            256   

In [10]:
# Reshape X_train and X_test to be compatible with the CNN input
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

In [11]:
import tensorflow as tf
from tensorflow.keras import layers

# Generator model
def build_generator(latent_dim):
    model = tf.keras.Sequential()
    model.add(layers.Dense(128, input_dim=latent_dim, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(X_train.shape[1], activation='tanh'))  # Output size must match the input shape
    return model

# Discriminator model
def build_discriminator(input_shape):
    model = tf.keras.Sequential()
    model.add(layers.Dense(512, input_shape=input_shape, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [12]:
# Define GAN parameters
latent_dim = 100
input_shape = (X_train.shape[1],)

# Build and compile the discriminator
discriminator = build_discriminator(input_shape)
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Build the generator
generator = build_generator(latent_dim)

# Combined GAN model
z = layers.Input(shape=(latent_dim,))
generated_data = generator(z)
discriminator.trainable = False
validity = discriminator(generated_data)

gan = tf.keras.Model(z, validity)
gan.compile(optimizer='adam', loss='binary_crossentropy')

In [13]:
def train_gan(generator, discriminator, gan, epochs=5000, batch_size=64):
    for epoch in range(epochs):
        # Train discriminator on real data
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_data = X_train[idx]

        # Generate fake data
        z = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_data = generator.predict(z)

        # Labels for real and fake data
        real_labels = np.ones((batch_size, 1))
        fake_labels = np.zeros((batch_size, 1))

        # Train the discriminator
        d_loss_real = discriminator.train_on_batch(real_data, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_data, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train generator (to fool the discriminator)
        z = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_labels = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(z, valid_labels)
        print(epoch)
        if epoch % 1000 == 0:
            print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")

# Train the GAN
train_gan(generator, discriminator, gan)

Streaming output truncated to the last 5000 lines.
2/2 [==============================] - 0s 4ms/step
2501
2/2 [==============================] - 0s 4ms/step
2502
2/2 [==============================] - 0s 4ms/step
2503
2/2 [==============================] - 0s 4ms/step
2504
2/2 [==============================] - 0s 4ms/step
2505
2/2 [==============================] - 0s 3ms/step
2506
2/2 [==============================] - 0s 4ms/step
2507
2/2 [==============================] - 0s 4ms/step
2508
2/2 [==============================] - 0s 4ms/step
2509
2/2 [==============================] - 0s 4ms/step
2510
2/2 [==============================] - 0s 3ms/step
2511
2/2 [==============================] - 0s 3ms/step
2512
2/2 [==============================] - 0s 4ms/step
2513
2/2 [==============================] - 0s 4ms/step
2514
2/2 [==============================] - 0s 4ms/step
2515
2/2 [==============================] - 0s 4ms/step
2516
2/2 [==============================] - 0s 4ms/step
25

In [14]:
# Generate synthetic data
def generate_synthetic_data(generator, num_samples):
    z = np.random.normal(0, 1, (num_samples, latent_dim))
    synthetic_data = generator.predict(z)
    return synthetic_data

# Generate synthetic S, V, and F beats
synthetic_S_beats = generate_synthetic_data(generator, 500)
synthetic_V_beats = generate_synthetic_data(generator, 500)
synthetic_F_beats = generate_synthetic_data(generator, 500)

# Expand dimensions to match X_train
synthetic_S_beats = np.expand_dims(synthetic_S_beats, axis=-1)
synthetic_V_beats = np.expand_dims(synthetic_V_beats, axis=-1)
synthetic_F_beats = np.expand_dims(synthetic_F_beats, axis=-1)

# Combine synthetic data with original training set
X_train_balanced = np.vstack([X_train, synthetic_S_beats, synthetic_V_beats, synthetic_F_beats])

# Convert y_train back to class labels
y_train_labels = np.argmax(y_train, axis=1)

# Create labels for synthetic data
y_synthetic_S = np.full((synthetic_S_beats.shape[0],), 1)
y_synthetic_V = np.full((synthetic_V_beats.shape[0],), 2)
y_synthetic_F = np.full((synthetic_F_beats.shape[0],), 3)

# Combine labels
y_train_balanced = np.concatenate([y_train_labels, y_synthetic_S, y_synthetic_V, y_synthetic_F])

# Convert labels to one-hot encoding
y_train_balanced = to_categorical(y_train_balanced, num_classes=5)

16/16 [==============================] - 0s 2ms/step


In [17]:
# Train the CNN-RNN model with the balanced dataset (real + synthetic data)
model, y_preds = CNN_RNN(X_train_balanced, y_train_balanced, X_test, y_test, 'categorical_crossentropy', -1)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs_cnn (InputLayer)     [(None, 186, 1)]          0         
                                                                 
 conv1d_6 (Conv1D)           (None, 181, 64)           448       
                                                                 
 batch_normalization_9 (Bat  (None, 181, 64)           256       
 chNormalization)                                                
                                                                 
 max_pooling1d_6 (MaxPoolin  (None, 91, 64)            0         
 g1D)                                                            
                                                                 
 conv1d_7 (Conv1D)           (None, 89, 64)            12352     
                                                                 
 batch_normalization_10 (Ba  (None, 89, 64)            256 

In [18]:
model.save("Hybrid_Multiclass.h5")

In [19]:
import tensorflow as tf

# Load the model
model = tf.keras.models.load_model("Hybrid_Multiclass.h5")

In [20]:
run_model = tf.function(lambda x: model(x))
# This is important, let's fix the input size.
BATCH_SIZE = 1
STEPS = 186
INPUT_SIZE = 1
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], model.inputs[0].dtype))

# model directory.
MODEL_DIR = "keras_lstm"
model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
tflite_model = converter.convert()
open("Hybrid_Multiclass.tflite", "wb").write(tflite_model)

472028